In [64]:
import re
import math
import warnings
import tqdm
import matplotlib
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from matplotlib.pyplot import MultipleLocator
# from lightgbm import LGBMClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")


%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

In [65]:
# ! conda install -c conda-forge sklearn-contrib-lightning
# ! conda install -c conda-forge tpot-imblearn
# ! conda install lightgbm
# ! conda install openpyxl

In [66]:
data1=pd.read_excel('/root/GDUT/表1-患者列表及临床信息.xlsx')
data2=pd.read_excel('/root/GDUT/表2-患者影像信息血肿及水肿的体积及位置.xlsx')
data3=pd.read_excel('/root/GDUT/表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx')
data4=pd.read_excel('/root/GDUT/表4-答案文件.xlsx')
data5=pd.read_excel('1b数据.xlsx').drop(['ID'], axis=1)

data1.rename(columns={data1.columns[0]: "ID"}, inplace=True)
data2.rename(columns={data2.columns[0]: "ID"}, inplace=True)

data_f_1=pd.read_excel('/root/GDUT/附表1-检索表格-流水号vs时间.xlsx')
data_kz=pd.read_excel('/root/GDUT/q11/扩张及时间.xlsx')


In [67]:

# 转换独热编码
xueya_list = []
for i in data5['血压']:
    y1 = i.split('/')[0]
    y2 = i.split('/')[1]
    if int(y1) > 139 or int(y1) < 90:
        status_i = 0
    elif int(y2) > 89 or int(y2) < 60:
        status_i = 0
    else:
        status_i = 1
    xueya_list.append(status_i)
data5['血压'] = np.array(xueya_list)
data5['性别_男'] = data5['性别'].apply(lambda x: 1 if x == '男' else 0)
data5['性别_女'] = data5['性别'].apply(lambda x: 1 if x == '女' else 0)
data5 = data5.drop(columns=['性别'])
x = data5['年龄'].values
scaler = MinMaxScaler()
x_normalized = scaler.fit_transform(x.reshape(-1, 1))
binarizer = Binarizer(threshold=0.5)
x_binarized = binarizer.fit_transform(x_normalized)
data5['年龄'] = x_binarized
data5 = data5.drop(['流水号'], axis = 1)

In [68]:
# 归一化
# 比例变量变数值变量，归一化的是数值变量
# 假设data5是你的数据框
hm_columns = data5.filter(like='HM_').columns.tolist()
ed_columns = data5.filter(like='ED_').columns.tolist()
hm_columns.pop(0)
ed_columns.pop(0)
# print(ed_columns)

# 获取'HM_volume'和'ED_volume.0'列的值
hm_volume = data5['HM_volume']
ed_volume = data5['ED_volume.0']

# 将以'HM_'开头的列与'HM_volume'相乘
data5[hm_columns] = data5[hm_columns].mul(hm_volume, axis=0)

# 将以'ED_'开头的列与'ED_volume.0'相乘
data5[ed_columns] = data5[ed_columns].mul(ed_volume, axis=0)
# 创建 MinMaxScaler 对象
scaler = MinMaxScaler()
# 选择要归一化的列（除了 '年龄' 列之外的所有列）
columns_to_normalize = [col for col in data5.columns if col != '性别']
# 使用 MinMaxScaler 对所选列进行归一化
data5[columns_to_normalize] = scaler.fit_transform(data5[columns_to_normalize])
train = data5[:100]
label = data_kz['是否扩张'].values[:100]
predict = data5[100:]
sm = SMOTE(random_state=0)
xres, yres = sm.fit_resample(train.values, label)

In [69]:
data5

,年龄,脑出血前mRS评分,高血压病史,卒中病史,糖尿病史,房颤史,冠心病史,吸烟史,饮酒史,发病到首次影像检查时间间隔,...,NCCT_original_firstorder_Median.Hemo,NCCT_original_firstorder_Minimum.Hemo,NCCT_original_firstorder_Range.Hemo,NCCT_original_firstorder_RobustMeanAbsoluteDeviation.Hemo,NCCT_original_firstorder_RootMeanSquared.Hemo,NCCT_original_firstorder_Skewness.Hemo,NCCT_original_firstorder_Uniformity.Hemo,NCCT_original_firstorder_Variance.Hemo,性别_男,性别_女
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094737,...,0.794741,0.580976,0.461355,0.596541,0.747897,0.073091,0.233654,0.575522,0.0,1.0
1,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.115789,...,0.312688,0.302489,0.415018,0.784219,0.401521,0.432276,0.062318,0.678884,1.0,0.0
2,1.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.073684,...,0.264387,0.173455,0.481753,0.421848,0.296407,0.379049,0.338820,0.323005,1.0,0.0
3,1.0,0.666667,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.031579,...,0.165330,0.145162,0.841963,0.543010,0.239052,0.550780,0.631206,0.475441,1.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,...,0.406462,0.494777,0.314800,0.505594,0.440597,0.383662,0.068760,0.401649,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,1.0,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.221053,...,0.606803,0.322885,0.524279,0.619519,0.577253,0.164114,0.277650,0.558400,0.0,1.0
156,0.0,0.666667,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.010526,...,0.235845,0.492805,0.332350,0.645449,0.373741,0.577202,0.118778,0.521251,1.0,0.0
157,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.107368,...,0.343452,0.357229,0.372732,0.559288,0.408873,0.436945,0.129904,0.439126,1.0,0.0
158,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.115789,...,0.196442,0.146322,0.385698,0.487636,0.229306,0.380374,0.221827,0.349817,1.0,0.0


In [70]:
import pickle

# 调用已保存的函数参数
with open('xgboost_default_params.pkl', 'rb') as f:
    default_params = pickle.load(f)
xgb_model = xgb.XGBClassifier(**default_params)
xgb_model.fit(xres,yres)
xgb_model_result = [round(i,4) for i in xgb_model.predict_proba(data5.values)[:,1]]

In [71]:
feature_importance_scores = xgb_model.feature_importances_
feature_importance_scores

array([0.00041293, 0.        , 0.04165813, 0.        , 0.        ,
       0.        , 0.        , 0.01295881, 0.        , 0.01033472,
       0.        , 0.        , 0.00463407, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00780626, 0.        ,
       0.00047247, 0.0058913 , 0.00323255, 0.        , 0.00410204,
       0.00072101, 0.00675062, 0.00844349, 0.00137912, 0.00212483,
       0.01972254, 0.00320556, 0.00653083, 0.        , 0.        ,
       0.02827577, 0.00343956, 0.0036412 , 0.00693185, 0.        ,
       0.02944231, 0.0326282 , 0.01278809, 0.01182895, 0.03514562,
       0.00033691, 0.00721952, 0.00039823, 0.        , 0.05365873,
       0.0036664 , 0.0008111 , 0.00547442, 0.        , 0.00499691,
       0.01169005, 0.01079739, 0.04132928, 0.01345471, 0.00745635,
       0.0073239 , 0.00491471, 0.00083908, 0.        , 0.00774792,
       0.03641348, 0.        , 0.        , 0.01570377, 0.00246452,
       0.        , 0.01150434, 0.01116846, 0.03053812, 0.08577

In [72]:
len(feature_importance_scores)

104

In [73]:
train.columns

Index(['年龄', '脑出血前mRS评分', '高血压病史', '卒中病史', '糖尿病史', '房颤史', '冠心病史', '吸烟史', '饮酒史',
       '发病到首次影像检查时间间隔',
       ...
       'NCCT_original_firstorder_Median.Hemo',
       'NCCT_original_firstorder_Minimum.Hemo',
       'NCCT_original_firstorder_Range.Hemo',
       'NCCT_original_firstorder_RobustMeanAbsoluteDeviation.Hemo',
       'NCCT_original_firstorder_RootMeanSquared.Hemo',
       'NCCT_original_firstorder_Skewness.Hemo',
       'NCCT_original_firstorder_Uniformity.Hemo',
       'NCCT_original_firstorder_Variance.Hemo', '性别_男', '性别_女'],
      dtype='object', length=104)

In [74]:
important_columns = []
for i,j in zip(list(feature_importance_scores), list(train.columns)):
    if i > 0.01:
        important_columns.append(j)
important_columns

['高血压病史',
 '吸烟史',
 '发病到首次影像检查时间间隔',
 'ED_ACA_R_Ratio',
 'ED_ACA_L_Ratio',
 'original_shape_Elongation.ED',
 'original_shape_Flatness.ED',
 'original_shape_LeastAxisLength.ED',
 'original_shape_MajorAxisLength.ED',
 'original_shape_Maximum2DDiameterColumn.ED',
 'original_shape_MinorAxisLength.ED',
 'NCCT_original_firstorder_90Percentile.ED',
 'NCCT_original_firstorder_Energy.ED',
 'NCCT_original_firstorder_Entropy.ED',
 'NCCT_original_firstorder_InterquartileRange.ED',
 'NCCT_original_firstorder_Range.ED',
 'NCCT_original_firstorder_Skewness.ED',
 'original_shape_Elongation.Hemo',
 'original_shape_Flatness.Hemo',
 'original_shape_LeastAxisLength.Hemo',
 'original_shape_MajorAxisLength.Hemo',
 'original_shape_Maximum2DDiameterColumn.Hemo',
 'original_shape_Maximum2DDiameterSlice.Hemo',
 'original_shape_Maximum3DDiameter.Hemo',
 'original_shape_MinorAxisLength.Hemo',
 'original_shape_Sphericity.Hemo',
 'original_shape_SurfaceArea.Hemo',
 'original_shape_SurfaceVolumeRatio.Hemo',
 'NCCT_or

In [75]:
data5[important_columns] # 已归一化

,高血压病史,吸烟史,发病到首次影像检查时间间隔,ED_ACA_R_Ratio,ED_ACA_L_Ratio,original_shape_Elongation.ED,original_shape_Flatness.ED,original_shape_LeastAxisLength.ED,original_shape_MajorAxisLength.ED,original_shape_Maximum2DDiameterColumn.ED,...,original_shape_Maximum2DDiameterSlice.Hemo,original_shape_Maximum3DDiameter.Hemo,original_shape_MinorAxisLength.Hemo,original_shape_Sphericity.Hemo,original_shape_SurfaceArea.Hemo,original_shape_SurfaceVolumeRatio.Hemo,NCCT_original_firstorder_Entropy.Hemo,NCCT_original_firstorder_Maximum.Hemo,NCCT_original_firstorder_Minimum.Hemo,NCCT_original_firstorder_Uniformity.Hemo
0,0.0,0.0,0.094737,0.252886,0.000000,0.690562,0.423493,0.534717,0.486735,0.545411,...,0.337284,0.437047,0.501765,0.811013,0.226877,0.012786,0.760605,0.495179,0.580976,0.233654
1,1.0,0.0,0.115789,0.221394,0.000000,0.890860,0.598458,0.470516,0.321530,0.525105,...,0.285795,0.375695,0.444583,0.627124,0.214643,0.080025,0.907789,0.365019,0.302489,0.062318
2,1.0,0.0,0.073684,0.251841,0.000000,0.863232,0.517890,0.603919,0.477446,0.636252,...,0.419727,0.616034,0.577782,0.330598,0.413949,0.095228,0.579144,0.387395,0.173455,0.338820
3,1.0,0.0,0.031579,0.021992,0.550946,0.300077,0.271812,0.412588,0.494905,0.429172,...,0.504225,0.536291,0.377542,0.291485,0.267849,0.206054,0.294472,0.716242,0.145162,0.631206
4,0.0,0.0,0.200000,0.100176,0.212439,0.515068,0.367420,0.690322,0.704448,0.973369,...,0.198405,0.208007,0.320355,0.605703,0.096407,0.246327,0.903870,0.330957,0.494777,0.068760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,1.0,0.0,0.221053,0.000000,0.185752,0.870203,0.692286,0.365403,0.208962,0.348064,...,0.907957,0.927352,0.524099,0.624086,0.156905,0.136628,0.707585,0.473792,0.322885,0.277650
156,1.0,1.0,0.010526,0.000000,0.000000,0.415720,0.112871,0.163302,0.287878,0.268813,...,0.226420,0.235189,0.254764,0.839556,0.085766,0.158936,0.878265,0.346794,0.492805,0.118778
157,1.0,0.0,0.107368,0.008170,0.000000,0.845392,0.999991,0.579515,0.253165,0.384828,...,0.206526,0.262991,0.297414,0.417810,0.155249,0.249705,0.824117,0.342428,0.357229,0.129904
158,1.0,1.0,0.115789,0.000000,0.118977,0.502884,0.355646,0.418597,0.422674,0.535853,...,0.300718,0.320687,0.466149,0.613606,0.246960,0.062251,0.718837,0.288904,0.146322,0.221827
